In [1]:
import pandas as pd
import matplotlib.pyplot as plt 
import warnings

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)

df = pd.read_csv('General_Files/train.csv')
df = df.drop(['Id'],axis=1)
print('\nDataframe de Treinamento: train.csv ({} linhas e {} colunas)'.format(df.shape[0],df.shape[1]))



Dataframe de Treinamento: train.csv (1460 linhas e 80 colunas)


### Transforming MSSubClass to categorical data

In [2]:
before = df['MSSubClass'].dtype
df[ 'MSSubClass' ] = df['MSSubClass'].astype(str)
after = df[ 'MSSubClass' ].dtype
print('Type before convertion is {}. Now MSSubClass data type is {}'.format(before,after))

Type before convertion is int64. Now MSSubClass data type is object


### Transforming nan values to 'NA' data

In [3]:
NA_fields = ['BsmtExposure','GarageType','BsmtQual','BsmtFinType1','FireplaceQu','GarageFinish']

print('Here is the columns that must convert nan to NA value:')
for col in NA_fields:
    print('\t{}: {}'.format(col, df[col].sort_values().unique()))

df[NA_fields] = df[NA_fields].fillna('NA')
print('\nThese is after convertion:')
for col in NA_fields:
    print('\t{}: {}'.format(col, df[col].sort_values().unique()))


Here is the columns that must convert nan to NA value:
	BsmtExposure: ['Av' 'Gd' 'Mn' 'No' nan]
	GarageType: ['2Types' 'Attchd' 'Basment' 'BuiltIn' 'CarPort' 'Detchd' nan]
	BsmtQual: ['Ex' 'Fa' 'Gd' 'TA' nan]
	BsmtFinType1: ['ALQ' 'BLQ' 'GLQ' 'LwQ' 'Rec' 'Unf' nan]
	FireplaceQu: ['Ex' 'Fa' 'Gd' 'Po' 'TA' nan]
	GarageFinish: ['Fin' 'RFn' 'Unf' nan]

These is after convertion:
	BsmtExposure: ['Av' 'Gd' 'Mn' 'NA' 'No']
	GarageType: ['2Types' 'Attchd' 'Basment' 'BuiltIn' 'CarPort' 'Detchd' 'NA']
	BsmtQual: ['Ex' 'Fa' 'Gd' 'NA' 'TA']
	BsmtFinType1: ['ALQ' 'BLQ' 'GLQ' 'LwQ' 'NA' 'Rec' 'Unf']
	FireplaceQu: ['Ex' 'Fa' 'Gd' 'NA' 'Po' 'TA']
	GarageFinish: ['Fin' 'NA' 'RFn' 'Unf']
